## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import Google_API_KEY

# Configure gmaps API key
gmaps.configure(api_key=Google_API_KEY)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Wind Direction,Weather,Country,Date
0,0,Rikitea,-23.1203,-134.9692,80.28,74,24,9.44,30,few clouds,PF,2022-03-19 05:49:23
1,1,Mar Del Plata,-38.0023,-57.5575,50.88,70,0,5.01,271,clear sky,AR,2022-03-19 05:49:24
2,2,Puerto Ayora,-0.7393,-90.3518,77.88,80,24,4.81,158,few clouds,EC,2022-03-19 05:46:26
3,3,Port Alfred,-33.5906,26.8910,73.40,85,100,8.48,67,overcast clouds,ZA,2022-03-19 05:49:25
4,4,Kapaa,22.0752,-159.3190,78.78,75,40,14.97,50,scattered clouds,US,2022-03-19 05:49:25


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 65
What is the maximum temperature you would like for your trip? 85


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Wind Direction,Weather,Country,Date
0,0,Rikitea,-23.1203,-134.9692,80.28,74,24,9.44,30,few clouds,PF,2022-03-19 05:49:23
2,2,Puerto Ayora,-0.7393,-90.3518,77.88,80,24,4.81,158,few clouds,EC,2022-03-19 05:46:26
3,3,Port Alfred,-33.5906,26.8910,73.40,85,100,8.48,67,overcast clouds,ZA,2022-03-19 05:49:25
4,4,Kapaa,22.0752,-159.3190,78.78,75,40,14.97,50,scattered clouds,US,2022-03-19 05:49:25
16,16,Cape Town,-33.9258,18.4232,67.10,76,0,11.50,230,clear sky,ZA,2022-03-19 05:46:25
17,17,Hermanus,-34.4187,19.2345,69.80,85,12,10.36,89,few clouds,ZA,2022-03-19 05:49:30
27,27,Vaini,-21.2000,-175.2000,80.28,77,100,26.96,311,light rain,TO,2022-03-19 05:49:35
31,31,Bluff,-46.6000,168.3333,66.79,77,24,14.56,123,few clouds,NZ,2022-03-19 05:49:37
33,33,Cooma,-36.2333,149.1333,71.37,49,45,6.91,170,scattered clouds,AU,2022-03-19 05:45:07
34,34,Atlantis,-33.5667,18.4834,65.17,48,1,5.53,127,clear sky,ZA,2022-03-19 05:49:38


In [5]:
# 4a. Determine if there are any empty rows.

preferred_cities_df.notnull().sum()


City_ID           262
City              262
Lat               262
Lng               262
Max Temp          262
Humidity          262
Cloudiness        262
Wind Speed        262
Wind Direction    262
Weather           262
Country           259
Date              262
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
preferred_cities_df = preferred_cities_df.dropna()

In [9]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Weather", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Weather,Lat,Lng,Hotel Name
0,Rikitea,PF,80.28,few clouds,-23.1203,-134.9692,
2,Puerto Ayora,EC,77.88,few clouds,-0.7393,-90.3518,
3,Port Alfred,ZA,73.40,overcast clouds,-33.5906,26.8910,
4,Kapaa,US,78.78,scattered clouds,22.0752,-159.3190,
16,Cape Town,ZA,67.10,clear sky,-33.9258,18.4232,
17,Hermanus,ZA,69.80,few clouds,-34.4187,19.2345,
27,Vaini,TO,80.28,light rain,-21.2000,-175.2000,
31,Bluff,NZ,66.79,few clouds,-46.6000,168.3333,
33,Cooma,AU,71.37,scattered clouds,-36.2333,149.1333,
34,Atlantis,ZA,65.17,clear sky,-33.5667,18.4834,


In [10]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": Google_API_KEY
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
        
    except (IndexError): 
        print("No hotel found...skipping.")   
        

No hotel found...skipping.
No hotel found...skipping.
No hotel found...skipping.
No hotel found...skipping.
No hotel found...skipping.
No hotel found...skipping.
No hotel found...skipping.
No hotel found...skipping.
No hotel found...skipping.
No hotel found...skipping.
No hotel found...skipping.
No hotel found...skipping.
No hotel found...skipping.
No hotel found...skipping.
No hotel found...skipping.
No hotel found...skipping.
No hotel found...skipping.
No hotel found...skipping.
No hotel found...skipping.
No hotel found...skipping.
No hotel found...skipping.
No hotel found...skipping.
No hotel found...skipping.
No hotel found...skipping.
No hotel found...skipping.
No hotel found...skipping.
No hotel found...skipping.
No hotel found...skipping.


In [21]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df.notnull().sum()

City          259
Country       259
Max Temp      259
Weather       259
Lat           259
Lng           259
Hotel Name    259
dtype: int64

In [22]:
# 8a. Create the output File (CSV)
output_data_file = "../Weather_Database/WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file, index_label="City_ID")

In [23]:
hotel_df

,City,Country,Max Temp,Weather,Lat,Lng,Hotel Name
0,Rikitea,PF,80.28,few clouds,-23.1203,-134.9692,People ThankYou
2,Puerto Ayora,EC,77.88,few clouds,-0.7393,-90.3518,Finch Bay Galapagos Hotel
3,Port Alfred,ZA,73.40,overcast clouds,-33.5906,26.8910,The Halyards Hotel
4,Kapaa,US,78.78,scattered clouds,22.0752,-159.3190,Sheraton Kauai Coconut Beach Resort
16,Cape Town,ZA,67.10,clear sky,-33.9258,18.4232,"Radisson Blu Hotel Waterfront, Cape Town"
...,...,...,...,...,...,...,...
695,Iranshahr,IR,70.14,clear sky,27.2025,60.6848,نیک سرشت
701,Valenca,BR,70.86,scattered clouds,-13.3703,-39.0731,Hotel Galeão
705,Ilhabela,BR,78.08,broken clouds,-23.7781,-45.3581,Ilha Flat Hotel
706,Madaoua,NE,70.45,clear sky,14.0730,5.9600,


In [39]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt style="font-weight: bold;">Hotel Name</dt><dd>{Hotel Name}</dd>
<dt style="font-weight: bold;">City</dt><dd>{City}</dd>
<dt style="font-weight: bold;">Country</dt><dd>{Country}</dd>
<dt style="font-weight: bold;">Current Weather</dt><dd>{Weather}</dd>
<dt style="font-weight: bold;">Current Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]

In [40]:
# 11a. Add a marker layer for each city to the map. 
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=2.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# 11b. Display the figure


In [41]:
fig

Figure(layout=FigureLayout(height='420px'))